#Text mining with LSTM

Keras examples on Github: https://github.com/keras-team/keras/tree/master/examples

In [1]:
import tensorflow as tf
from tensorflow import keras
from keras.layers import LSTM
from keras.layers import  Bidirectional
import numpy as np

print(tf.__version__)

2.9.2


Import IMDB Data

In [2]:
imdb = keras.datasets.imdb

(train_data, train_labels), (test_data, test_labels) = imdb.load_data(num_words=10000)

17464789/17464789 [==============================] - 0s 0us/step


In [3]:
print("Training entries: {}, labels: {}".format(len(train_data), len(train_labels)))

Training entries: 25000, labels: 25000


Helper function to decode integers to words

In [4]:
# A dictionary mapping words to an integer index
word_index = imdb.get_word_index()

# The first indices are reserved
word_index = {k:(v+3) for k,v in word_index.items()} 
word_index["<PAD>"] = 0
word_index["<START>"] = 1
word_index["<UNK>"] = 2  # unknown
word_index["<UNUSED>"] = 3

reverse_word_index = dict([(value, key) for (key, value) in word_index.items()])

def decode_review(text):
    return ' '.join([reverse_word_index.get(i, '?') for i in text])

1641221/1641221 [==============================] - 0s 0us/step


## Data Prep
Padding vectors to be of same length

In [5]:
train_data = keras.preprocessing.sequence.pad_sequences(train_data,
                                                        value=word_index["<PAD>"],
                                                        padding='post',
                                                        maxlen=256)

test_data = keras.preprocessing.sequence.pad_sequences(test_data,
                                                       value=word_index["<PAD>"],
                                                       padding='post',
                                                       maxlen=256)

In [6]:
print(train_data.shape[1])

256


In [13]:
train_data[0]

array([   1,   14,   22,   16,   43,  530,  973, 1622, 1385,   65,  458,
       4468,   66, 3941,    4,  173,   36,  256,    5,   25,  100,   43,
        838,  112,   50,  670,    2,    9,   35,  480,  284,    5,  150,
          4,  172,  112,  167,    2,  336,  385,   39,    4,  172, 4536,
       1111,   17,  546,   38,   13,  447,    4,  192,   50,   16,    6,
        147, 2025,   19,   14,   22,    4, 1920, 4613,  469,    4,   22,
         71,   87,   12,   16,   43,  530,   38,   76,   15,   13, 1247,
          4,   22,   17,  515,   17,   12,   16,  626,   18,    2,    5,
         62,  386,   12,    8,  316,    8,  106,    5,    4, 2223, 5244,
         16,  480,   66, 3785,   33,    4,  130,   12,   16,   38,  619,
          5,   25,  124,   51,   36,  135,   48,   25, 1415,   33,    6,
         22,   12,  215,   28,   77,   52,    5,   14,  407,   16,   82,
          2,    8,    4,  107,  117, 5952,   15,  256,    4,    2,    7,
       3766,    5,  723,   36,   71,   43,  530,  4

In [17]:
print(decode_review(train_data[0]))

<START> this film was just brilliant casting location scenery story direction everyone's really suited the part they played and you could just imagine being there robert <UNK> is an amazing actor and now the same being director <UNK> father came from the same scottish island as myself so i loved the fact there was a real connection with this film the witty remarks throughout the film were great it was just brilliant so much that i bought the film as soon as it was released for <UNK> and would recommend it to everyone to watch and the fly fishing was amazing really cried at the end it was so sad and you know what they say if you cry at a film it must have been good and this definitely was also <UNK> to the two little boy's that played the <UNK> of norman and paul they were just brilliant children are often left out of the <UNK> list i think because the stars that play them all grown up are such a big profile for the whole film but these children are amazing and should be praised for wha

## Build LSTM Model

In [19]:
# input shape is the vocabulary count used for the movie reviews (10,000 words)
vocab_size = 10000
embedding_dim = 32
dropout_rate=0.1

model = keras.Sequential()
model.add(keras.layers.Embedding(vocab_size, embedding_dim, input_length=train_data.shape[1]))
model.add(keras.layers.LSTM(128, dropout=0.2, recurrent_dropout=0.2))
model.add(keras.layers.Dense(1, activation=tf.nn.sigmoid))

model.summary()

Model: "sequential_7"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_7 (Embedding)     (None, 256, 32)           320000    
                                                                 
 lstm_8 (LSTM)               (None, 128)               82432     
                                                                 
 dense_5 (Dense)             (None, 1)                 129       
                                                                 
Total params: 402,561
Trainable params: 402,561
Non-trainable params: 0
_________________________________________________________________


In [8]:
# CONV + LSTM

# input shape is the vocabulary count used for the movie reviews (10,000 words)
vocab_size = 10000
embedding_dim = 32
dropout_rate=0.1

model = keras.Sequential()
model.add(keras.layers.Embedding(vocab_size, embedding_dim, input_length=train_data.shape[1]))
model.add(keras.layers.Dropout(dropout_rate))
model.add(keras.layers.Conv1D(128 ,kernel_size= 2 , activation= "relu" , strides= 1 , padding= "same" ))
model.add(keras.layers.LSTM(128, dropout=0.2, recurrent_dropout=0.2))
model.add(keras.layers.Dense(1, activation=tf.nn.sigmoid))


model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_1 (Embedding)     (None, 256, 32)           320000    
                                                                 
 dropout_1 (Dropout)         (None, 256, 32)           0         
                                                                 
 conv1d (Conv1D)             (None, 256, 128)          8320      
                                                                 
 lstm_1 (LSTM)               (None, 128)               131584    
                                                                 
 dense_1 (Dense)             (None, 10000)             1290000   
                                                                 
Total params: 1,749,904
Trainable params: 1,749,904
Non-trainable params: 0
_________________________________________________________________


In [27]:
# model.add(keras.layers.GlobalAveragePooling1D())

# LSTM + CONV

# input shape is the vocabulary count used for the movie reviews (10,000 words)
vocab_size = 10000
embedding_dim = 32
dropout_rate=0.1

model = keras.Sequential()
model.add(keras.layers.Embedding(vocab_size, embedding_dim, input_length=train_data.shape[1]))
model.add(keras.layers.Dropout(dropout_rate))
model.add(keras.layers.LSTM(128, dropout=0.2, recurrent_dropout=0.2))
model.add(keras.layers.Conv1D(128 ,kernel_size= 2 , activation= "relu" , strides= 1 , padding= "same" ))
model.add(keras.layers.Dense(1, activation=tf.nn.sigmoid))

model.summary()

Model: "sequential_15"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_15 (Embedding)    (None, 256, 32)           320000    
                                                                 
 dropout_11 (Dropout)        (None, 256, 32)           0         
                                                                 
 lstm_16 (LSTM)              (None, 256, 128)          82432     
                                                                 
 conv1d_9 (Conv1D)           (None, 256, 128)          32896     
                                                                 
 global_average_pooling1d_2   (None, 128)              0         
 (GlobalAveragePooling1D)                                        
                                                                 
 dense_10 (Dense)            (None, 1)                 129       
                                                     

In [29]:
# Two LSTM Layers
# input shape is the vocabulary count used for the movie reviews (10,000 words)
vocab_size = 10000
embedding_dim = 32
dropout_rate=0.1

model = keras.Sequential()
model.add(keras.layers.Embedding(vocab_size, embedding_dim, input_length=train_data.shape[1]))
model.add(keras.layers.Dropout(dropout_rate))
model.add(keras.layers.LSTM(128, dropout=0.2, recurrent_dropout=0.2,return_sequences=True))
model.add(keras.layers.LSTM(128, dropout=0.2, recurrent_dropout=0.2))
model.add(keras.layers.Dense(1, activation=tf.nn.sigmoid))


model. compile (optimizer= "adam" , loss= 'mae', metrics=[ 'accuracy'])
model.summary()

Model: "sequential_17"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_17 (Embedding)    (None, 256, 32)           320000    
                                                                 
 dropout_13 (Dropout)        (None, 256, 32)           0         
                                                                 
 lstm_19 (LSTM)              (None, 256, 128)          82432     
                                                                 
 lstm_20 (LSTM)              (None, 256, 128)          131584    
                                                                 
 dense_12 (Dense)            (None, 256, 1)            129       
                                                                 
Total params: 534,145
Trainable params: 534,145
Non-trainable params: 0
_________________________________________________________________


In [ ]:
# what if we want to predict the next word? 
# Two LSTM Layers
# input shape is the vocabulary count used for the movie reviews (10,000 words)
vocab_size = 10000
embedding_dim = 32
dropout_rate=0.1

model = keras.Sequential()
model.add(keras.layers.Embedding(vocab_size, embedding_dim, input_length=train_data.shape[1]))
model.add(keras.layers.Dropout(dropout_rate))
model.add(keras.layers.LSTM(128, dropout=0.2, recurrent_dropout=0.2,return_sequences=True))
model.add(keras.layers.LSTM(128, dropout=0.2, recurrent_dropout=0.2,return_sequences=True ))
model.add(keras.layers.Dense(10000, activation=tf.nn.softmax))


Compile

In [ ]:
model.compile(optimizer=keras.optimizers.Adam(),
              loss='binary_crossentropy',
              metrics=['accuracy'])

Validation Data

In [ ]:
x_val = train_data[:10000]
partial_x_train = train_data[10000:]

y_val = train_labels[:10000]
partial_y_train = train_labels[10000:]

Training

In [ ]:
history = model.fit(partial_x_train,
                    partial_y_train,
                    epochs=2,
                    batch_size=32,
                    validation_data=(x_val, y_val),
                    verbose=1)

Epoch 1/2
 18/469 [>.............................] - ETA: 2:59 - loss: 0.6945 - accuracy: 0.4844

KeyboardInterrupt: ignored

##Bidirectional LSTM model

In [ ]:
# input shape is the vocabulary count used for the movie reviews (10,000 words)

vocab_size = 10000
embedding_dim = 50
dropout_rate=0.1

model = keras.Sequential()
model.add(keras.layers.Embedding(vocab_size, embedding_dim, input_length=train_data.shape[1]))
model.add(keras.layers.Dropout(dropout_rate))
model.add(keras.layers.Bidirectional(keras.layers.LSTM(64)))
model.add(keras.layers.Dropout(dropout_rate))
model.add(keras.layers.Dense(1, activation=tf.nn.sigmoid))

model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_1 (Embedding)     (None, 256, 50)           500000    
                                                                 
 dropout_1 (Dropout)         (None, 256, 50)           0         
                                                                 
 bidirectional (Bidirectiona  (None, 128)              58880     
 l)                                                              
                                                                 
 dropout_2 (Dropout)         (None, 128)               0         
                                                                 
 dense_1 (Dense)             (None, 1)                 129       
                                                                 
Total params: 559,009
Trainable params: 559,009
Non-trainable params: 0
________________________________________________

In [ ]:
model.compile(optimizer=tf.train.AdamOptimizer(),
              loss='binary_crossentropy',
              metrics=['accuracy'])

In [ ]:
history = model.fit(partial_x_train,
                    partial_y_train,
                    epochs=2,
                    batch_size=32,
                    validation_data=(x_val, y_val),
                    verbose=1)

Train on 15000 samples, validate on 10000 samples
Epoch 1/2
15000/15000 [==============================] - 287s 19ms/step - loss: 0.5292 - acc: 0.7327 - val_loss: 0.4652 - val_acc: 0.7946
Epoch 2/2
15000/15000 [==============================] - 286s 19ms/step - loss: 0.3466 - acc: 0.8627 - val_loss: 0.4133 - val_acc: 0.8295
